In [1]:
from pathlib import Path
from pprint import pprint

import pandas as pd

import graphrag.api as api
from graphrag.config.load_config import load_config
from graphrag.index.typing.pipeline_run_result import PipelineRunResult

### handson ディレクトリ概要
- `.env`: Graphrag 実行時の環境変数を配置する。
- `settings.yaml`: Graphrag の設定ファイル。本演習のパラメータが定義されている。
- `input/`: インデックス化対象となる元ドキュメントを格納する。
- `output/`: build_index 実行後に作成されるエンティティやリレーションなどの成果物が保存される。
- `cache/`・`logs/`: 実行時のキャッシュやログを保持し、再実行時の高速化やトラブルシュートに使う。
- `prompts/`: 追加でカスタムプロンプトを定義したい場合に配置する。

このセルでは `handson` ディレクトリにある `settings.yaml` から設定を読み込み、`graphrag.api.build_index` を実行してインデックスを構築します。各ワークフローの成功／失敗が標準出力に表示されるため、実行後にログを確認して次の処理（データの可視化や検索）へ進みます。

In [2]:

PROJECT_DIRECTORY = "./handson"
graphrag_config = load_config(Path(PROJECT_DIRECTORY))

Model config based on fnllm is deprecated and will be removed in GraphRAG v3, please use chat or embedding instead to switch to LiteLLM config.
Model config based on fnllm is deprecated and will be removed in GraphRAG v3, please use chat or embedding instead to switch to LiteLLM config.


In [3]:
index_result: list[PipelineRunResult] = await api.build_index(config=graphrag_config)

for workflow_result in index_result:
    status = f"error\n{workflow_result.errors}" if workflow_result.errors else "success"
    print(f"Workflow Name: {workflow_result.workflow}\tStatus: {status}")

Workflow Name: load_input_documents	Status: success
Workflow Name: create_base_text_units	Status: success
Workflow Name: create_final_documents	Status: success
Workflow Name: extract_graph	Status: success
Workflow Name: finalize_graph	Status: success
Workflow Name: extract_covariates	Status: success
Workflow Name: create_communities	Status: success
Workflow Name: create_final_text_units	Status: success
Workflow Name: create_community_reports	Status: success
Workflow Name: generate_text_embeddings	Status: success


In [4]:
entities_df = pd.read_parquet(f"{PROJECT_DIRECTORY}/output/entities.parquet")
communities_df = pd.read_parquet(f"{PROJECT_DIRECTORY}/output/communities.parquet")
community_reports_df = pd.read_parquet(
    f"{PROJECT_DIRECTORY}/output/community_reports.parquet"
)
relationships_df = pd.read_parquet(f"{PROJECT_DIRECTORY}/output/relationships.parquet")
documents_df = pd.read_parquet(f"{PROJECT_DIRECTORY}/output/documents.parquet")
text_units_df = pd.read_parquet(f"{PROJECT_DIRECTORY}/output/text_units.parquet")

#### Entity カラム概要
- `id`: 解析パイプライン内でユニークなエンティティ ID（UUID）。
- `human_readable_id`: 連番の簡易 ID。グラフ表示やデバッグに便利。
- `title`: エンティティ名。後続の可視化やクエリでノードラベルとして利用。
- `type`: LLM が推定したカテゴリ（PERSON/ORGANIZATION など）。
- `description`: エンティティの説明文。検索応答の根拠として活用。
- `text_unit_ids`: このエンティティが登場するテキストユニット ID のリスト。
- `frequency`: エンティティが参照された回数。重要度の簡易スコアとして使える。
- `degree`: グラフ上で接続している関係数。ハブノードの特定に有用。
- `x`, `y`: レイアウト計算済みの座標。ノートブック内でグラフを描画する際に使用。

In [5]:
entities_df

,id,human_readable_id,title,type,description,text_unit_ids,frequency,degree,x,y
0,135fda80-1010-45d9-85ca-2a7f3297099e,0,MICROSOFT FOUNDRY,"組織,プラットフォーム,サービス",Microsoft Foundryは、Azure上で提供されるプロジェクト管理・モデルデプロ...,[3c75940ca8f9625dc411579579ad573a326d8e295b1cc...,1,3,0.0,0.0
1,53370b69-0cc0-4192-9a80-dd4905e9bc5d,1,AZURE,"クラウドサービス,プラットフォーム,組織",AzureはMicrosoftが提供するクラウドプラットフォームであり、Foundry Mo...,[3c75940ca8f9625dc411579579ad573a326d8e295b1cc...,1,2,0.0,0.0
2,bdbabe2f-5d21-4656-8130-cde17da3a85a,2,FOUNDRY MODELS,"AIモデル,技術,製品",Foundry Modelsは、Azureによって直接販売されるAIモデル群であり、Micr...,[3c75940ca8f9625dc411579579ad573a326d8e295b1cc...,1,7,0.0,0.0
3,1c219012-bed8-4b46-9a4d-4aca591732b7,3,FOUNDRY AGENT SERVICE,"サービス,技術",Foundry Agent Serviceは、Azure OpenAIモデルのサポートを提供...,[3c75940ca8f9625dc411579579ad573a326d8e295b1cc...,1,2,0.0,0.0
4,bbbe2bff-7c05-41c4-926d-29da1c52c9bc,4,FOUNDRY リソース,"リソース,技術",Foundryリソースは、Foundryプロジェクトの構築基盤となるリソースであり、標準デプ...,[3c75940ca8f9625dc411579579ad573a326d8e295b1cc...,1,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
387,60285f82-dff8-430f-9f2e-1a714a41b2a1,387,AZURE OPENAI モデルの微調整,"プロセス,技術,情報源",Azure OpenAIモデルの微調整は、Azure上でAIモデルをカスタマイズ・最適化する...,[1fa26d7c50d101f5b752b2495651df17bbd516e58082a...,1,1,0.0,0.0
388,f1cf232a-339c-48cc-91c7-aa452a683087,388,AI,"技術,分野",AI（人工知能）は、この記事の作成支援やモデル運用に関わる技術分野です。,[1fa26d7c50d101f5b752b2495651df17bbd516e58082a...,1,2,0.0,0.0
389,8796e104-09d1-4bb7-978f-dfdf1406ef5e,389,作成者,"人物,役割",作成者は、AIの支援を受けてこの記事を執筆した人物です。,[1fa26d7c50d101f5b752b2495651df17bbd516e58082a...,1,2,0.0,0.0
390,65e6db43-c398-474e-95e7-131d98b735fe,390,記事,"ドキュメント,情報源",記事は、モデルの廃止やAzure OpenAIに関する情報をまとめた文書です。,[1fa26d7c50d101f5b752b2495651df17bbd516e58082a...,1,3,0.0,0.0


#### Relationships カラム概要
- `id`: 関係エッジのユニーク ID（UUID）。
- `human_readable_id`: 連番で付与された簡易 ID。
- `source` / `target`: エッジが結ぶエンティティのタイトル。`entities_df['title']` と対応。
- `description`: LLM が生成した関係の説明文。
- `weight`: 関係の重要度スコア。GraphWidget の線幅などに利用。
- `combined_degree`: 接続するノード双方の次数から計算された指標。ハブ間の関係を把握しやすい。
- `text_unit_ids`: 関係が出現したテキストユニット ID の一覧。根拠ドキュメント追跡に使える。

In [6]:
relationships_df

,id,human_readable_id,source,target,description,weight,combined_degree,text_unit_ids
0,2e8d0faf-ccb6-4935-a1c3-7982317ea5d6,0,MICROSOFT FOUNDRY,AZURE,Microsoft FoundryはAzure上で運用されており、Azureによる直接販売や...,9.0,5,[3c75940ca8f9625dc411579579ad573a326d8e295b1cc...
1,c3a28688-1f19-4861-8db6-565c1385e554,1,MICROSOFT FOUNDRY,FOUNDRY MODELS,Foundry ModelsはMicrosoft Foundryプラットフォーム上で利用可能...,9.0,10,[3c75940ca8f9625dc411579579ad573a326d8e295b1cc...
2,c326ecb7-dd83-49bd-a6ee-b2fbe5e2c8cd,2,MICROSOFT FOUNDRY,FOUNDRY AGENT SERVICE,Foundry Agent ServiceはMicrosoft Foundryの一部サービス...,8.0,5,[3c75940ca8f9625dc411579579ad573a326d8e295b1cc...
3,a54c581c-65c9-49ac-87a5-93f3541f6b2e,3,AZURE,AZURE OPENAI,Azure OpenAIはAzureクラウド上で運用されているAIサービスであるため、両者は...,9.0,22,[3c75940ca8f9625dc411579579ad573a326d8e295b1cc...
4,64b3898e-13d3-43fe-806f-cdea9eacaf78,4,FOUNDRY MODELS,デプロイ オプション,Foundry Modelsは複数のデプロイオプション（標準デプロイ、マネージドAPI、サー...,8.0,8,[3c75940ca8f9625dc411579579ad573a326d8e295b1cc...
...,...,...,...,...,...,...,...,...
1087,653492da-295a-4014-8c1f-27ccdf91a5f5,1087,AZURE OPENAI,AZURE OPENAI モデルの微調整,Azure OpenAIモデルの微調整は、Azure OpenAIサービス上でモデルをカスタ...,16.0,21,[1fa26d7c50d101f5b752b2495651df17bbd516e58082a...
1088,43d9b34d-0a51-498c-aef7-fd21153700bb,1088,AI,作成者,作成者はAIの支援を受けて記事を執筆したため、両者は執筆プロセスで関連している。,14.0,4,[1fa26d7c50d101f5b752b2495651df17bbd516e58082a...
1089,4bf8fc0b-f11b-4452-ac59-908dd8f3ef00,1089,AI,記事,記事はAIの支援を受けて執筆されたため、両者は執筆プロセスで関連している。,2.0,5,[1fa26d7c50d101f5b752b2495651df17bbd516e58082a...
1090,d630652d-308a-45d5-aee6-8ac5c1e91a4c,1090,作成者,記事,記事は作成者によって執筆されたため、両者は執筆プロセスで関連している。,14.0,5,[1fa26d7c50d101f5b752b2495651df17bbd516e58082a...


### Community Reports カラム概要
- `id`: コミュニティレポートのユニーク ID（UUID）。
- `human_readable_id`: 連番で付いた簡易 ID。
- `community`: レポート対象コミュニティの ID。`communities_df['id']` と対応。
- `level`: コミュニティの階層レベル。0（グローバル）～ N。
- `parent` / `children`: 階層構造を表す親コミュニティ ID と子コミュニティ ID のリスト。
- `title`: レポートの見出しとして使われる要約タイトル。
- `summary`: コミュニティ全体の要約文。
- `full_content`: 箇条書きやセクションを含む詳細レポート本文。
- `full_content_json`: `full_content` を JSON 構造化したバージョン。後続の加工や UI 連携で利用。
- `findings`: 主要な洞察・リスク・推奨事項を配列で保持。
- `rank`: コミュニティの重要度スコア。
- `rating_explanation`: ランク付けに関する説明文。
- `period`: 解析対象期間など、レポートに紐づく時間情報（存在すれば）。
- `size`: コミュニティに属するテキストユニット数。

In [7]:
community_reports_df

,id,human_readable_id,community,level,parent,children,title,summary,full_content,rank,rating_explanation,findings,full_content_json,period,size
0,4c13b01777d64983b9d6df5e33f24a0a,67,67,3,57,[],GPT-5とChat Completions APIの技術連携コミュニティ,本コミュニティは、GPT-5モデル、Chat Completions API、及びAPI利用...,# GPT-5とChat Completions APIの技術連携コミュニティ\n\n本コミ...,8.5,GPT-5、Chat Completions API、エラーコード400の関係性は、Azur...,[{'explanation': 'GPT-5はChat Completions APIを通...,"{\n ""title"": ""GPT-5とChat Completions APIの技術...",2025-12-03,2
1,de87fae183504a1595ba690a9399be11,68,68,3,57,[],Azure OpenAI GPT-5シリーズと登録プロセスのコミュニティ,このコミュニティは、Azure OpenAIサービス上で提供されるGPT-5シリーズ（GPT...,# Azure OpenAI GPT-5シリーズと登録プロセスのコミュニティ\n\nこのコミ...,9.2,GPT-5シリーズと登録プロセスの関係性は、Azure OpenAIプラットフォームの最新モ...,[{'explanation': 'GPT-5は、Azure OpenAIサービスで提供され...,"{\n ""title"": ""Azure OpenAI GPT-5シリーズと登録プロセス...",2025-12-03,4
2,f5e406966b324a42ad0fb8de872f38b7,69,69,3,57,[],GPT-5-MINIと最大出力トークン数のAzure OpenAIドキュメントコミュニティ,このコミュニティは、Azure OpenAIサービスにおけるGPT-5-MINIモデルとその...,# GPT-5-MINIと最大出力トークン数のAzure OpenAIドキュメントコミュニテ...,9.2,GPT-5-MINIと最大出力トークン数は、Azure OpenAIプラットフォームの最新モ...,[{'explanation': 'GPT-5-MINIはGPT-5シリーズの軽量版AIモデ...,"{\n ""title"": ""GPT-5-MINIと最大出力トークン数のAzure Op...",2025-12-03,2
3,18b5f33e79dd48668995635294426175,70,70,3,57,[],GPT-5シリーズ・O3-MINIとモデルテーブルのドキュメント連携コミュニティ,本コミュニティは、Azure OpenAIプラットフォームにおけるGPT-5シリーズおよびO...,# GPT-5シリーズ・O3-MINIとモデルテーブルのドキュメント連携コミュニティ\n\n...,8.5,モデルテーブルがGPT-5シリーズおよびO3-MINIの詳細情報・リージョン可用性を集約し、...,[{'explanation': 'モデルテーブルは、GPT-5シリーズ各モデルおよびO3-...,"{\n ""title"": ""GPT-5シリーズ・O3-MINIとモデルテーブルのドキュ...",2025-12-03,2
4,83d26d2b308b43b98b15c4d5bd2d3b93,39,39,2,17,[],GPT-4.1シリーズとAzure OpenAI技術ドキュメントコミュニティ,"本コミュニティは、GPT-4.1シリーズ（gpt-4.1, gpt-4.1-mini, gp...",# GPT-4.1シリーズとAzure OpenAI技術ドキュメントコミュニティ\n\n本コ...,9.2,GPT-4.1シリーズの技術仕様、モデル間の関連性、API・デプロイメントの詳細、ドキュメン...,[{'explanation': 'GPT-4.1シリーズは、gpt-4.1、gpt-4.1...,"{\n ""title"": ""GPT-4.1シリーズとAzure OpenAI技術ドキュ...",2025-12-03,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,66332a91913f4de993120918052fe89f,6,6,0,-1,"[26, 27]",Azure OpenAI 地域別AIモデル提供・ドキュメント連携コミュニティ,本コミュニティは、Azure OpenAIサービスにおけるAIモデル（GPT-5、GPT-4...,# Azure OpenAI 地域別AIモデル提供・ドキュメント連携コミュニティ\n\n本コ...,9.5,本レポートはAzure OpenAIのAIモデル提供状況と地域展開、ドキュメント連携の実態を...,[{'explanation': 'Azure OpenAIサービスにおけるAIモデル（GP...,"{\n ""title"": ""Azure OpenAI 地域別AIモデル提供・ドキュメン...",2025-12-03,29
67,6f75f671889844739c5cd2959d3b7075,7,7,0,-1,"[28, 29, 30, 31]",Azure OpenAIモデル・リージョン・機能連携コミュニティ,本コミュニティは、Azure OpenAIサービスにおけるAIモデル（GPT-4o、GPT-...,# Azure OpenAIモデル・リージョン・機能連携コミュニティ\n\n本コミュニティは...,9.5,AIモデル・リージョン・機能・技術文書の連携が網羅的に記載されており、Azure OpenA...,[{'explanation': '米国東部2、スウェーデン中部、米国中北部、スイス西部など...,"{\n ""title"": ""Azure OpenAIモデル・リージョン・機能連携コミュ...",2025-12-03,41
68,ce02bb546747495994239e63d9c0743b,8,8,0,-1,[],Azure OpenAI マルチモーダルモデル・ドキュメント コミュニティ,本コミュニティは、Azure OpenAIのマルチモーダルAIモデル（GPT-40、GPT-...,# Azure OpenAI マルチモーダルモデル・ドキュメント コミュニティ\n\n本コミ...,9.2,本レポートは、Azure OpenAIの主要マルチモーダルモデル群の技術的関係性と、ドキュメ...,[{'explanation': 'GPT-40は、テキストと画像の両方を入力として受け付け...,"{\n ""title"": ""Azure OpenAI マルチモーダルモデル・ドキュメン...",2025-12-03,6
69,4c56d9ba130d4796ba3d96ccba979a41,9,9,0,-1,"[32, 33, 34, 35]",Azure OpenAI GPT-5.1 & Oシリーズ技術ドキュメント・コミュニティ,本コミュニティは、Azure OpenAIプラットフォーム上でGPT-5.1シリーズおよびO...,# Azure OpenAI GPT-5.1 & Oシリーズ技術ドキュメント・コミュニティ\...,9.5,本レポートはAzure OpenAIの最新モデル群の技術仕様・運用・機能・アクセス制御・ドキ...,[{'explanation': 'GPT-5.1シリーズ（GPT-5.1、GPT-5.1-...,"{\n ""title"": ""Azure OpenAI GPT-5.1 & Oシリーズ技...",2025-12-03,54


## Local Search の仕組み

1. **ユーザーのクエリ**（＋過去の会話履歴）が入力される。  
2. クエリ内容に基づき、**知識グラフから意味的に関連する可能性のあるエンティティ**を抽出する（embedding 類似度などを利用）。  
3. 抽出したエンティティを起点に、以下の候補情報を収集する：  
   - 元ドキュメントのテキストチャンク（Entity-Text Units）  
   - 関関連エンティティや関係性（Entity-Entity / Relationships）  
   - エンティティ属性・メタデータ（Covariates）  
   - コミュニティ構造・レポート（Community Reports）  
4. 収集したチャンク・エンティティ・関係・レポートを**ランキングとフィルタリング**で精査し、使用する情報のみを選定する。  
5. 選ばれた情報（テキスト + 構造化データ）を **LLM に入力し、最終的な回答を生成**する。

> Local Search は、全文検索だけでなく **知識グラフの構造化情報とテキスト情報を組み合わせるハイブリッド手法**で、高精度なエンティティ中心検索を実現します。

参考：https://microsoft.github.io/graphrag/query/local_search/


In [ ]:
response, context = await api.local_search(
    config=graphrag_config,
    entities=entities_df,
    communities=communities_df,
    community_reports=community_reports_df,
    text_units=text_units_df,
    relationships=relationships_df,
    covariates=None,
    community_level=2,
    response_type="Multiple Paragraphs",
    query="Azure OpenAIで使えるモデルを教えて",
)

In [11]:
print(response)

## GPT-5.1の概要

GPT-5.1は、Azure Foundry Modelsの最新世代に属する大規模言語モデルであり、2025年11月13日にリリース予定です。GPT-5.1シリーズには、gpt-5.1、gpt-5.1-chat、gpt-5.1-codex、gpt-5.1-codex-miniなど複数のバリエーションが含まれています。これらのモデルは、米国東部2およびスウェーデン中部リージョンで利用可能で、グローバル標準およびDataZone標準として提供されます。利用には登録が必要な制限付きアクセスモデルとなっています [Data: Entities (31, 32, 85); Relationships (28, 80); Sources (2)]。

## 主な機能と特徴

GPT-5.1は、以下のような多様な機能を備えています。

- **推論**  
- **Chat Completions API**  
- **応答API**  
- **構造化出力**  
- **テキストおよび画像処理**  
- **関数・ツール・並列ツール呼び出し**  
- **最大コンテキストウィンドウ：400,000トークン**  
- **最大出力トークン数：128,000トークン**  
- **トレーニングデータ：2024年9月30日まで**  
これらの機能により、自然言語処理だけでなく、画像入力や複雑なツール連携、構造化データの出力など、幅広い用途に対応しています [Data: Entities (32, 85); Relationships (80); Sources (2)]。

## バリエーション

GPT-5.1シリーズには、用途に応じた複数のモデルが存在します。

- **gpt-5.1**：シリーズの中心モデル。最大400,000トークンのコンテキストウィンドウと128,000トークンの出力に対応。
- **gpt-5.1-chat**：チャット形式に特化し、最大128,000トークンのコンテキストウィンドウと16,384トークンの出力に対応。
- **gpt-5.1-codex / codex-mini**：主にコード生成やプログラミング支援に特化し、Codex CLIやVS Code拡張機能にも最適化されています [Data: En

## Global Search の仕組み

1. ドキュメント群からナレッジグラフを構築し、グラフ内のエンティティが属する **コミュニティ (Community)** を検出／クラスタリングする。  
2. 各コミュニティに対して、LLM による要約を作成し **Community Report（コミュニティ要約）** を生成する。  
3. ユーザーのクエリに対し、指定レベルのコミュニティ階層に属するすべての Community Report を対象に、Map-Reduce 方式で応答生成を試みる。  
   - Map工程：Community Report を一定サイズのテキストチャンクに分割 → 各チャンクごとに中間応答 (Intermediate Response) を生成  
   - Reduce工程：すべての中間応答をフィルタ／統合 → 最終的な応答を生成  
4. 必要に応じて、コミュニティ階層のレベル (抽象度) を調整 — 抽象度の低いレポートは情報量が多く、詳細な応答が可能だが、コストやトークン消費が増える。  
5. この手法により、ドキュメント全体を俯瞰した **広範なテーマ、全体構造、総合的傾向** への問いに対して、一貫性のある総合的な回答を返すことが可能となる。  

参考：https://microsoft.github.io/graphrag/query/global_search/

In [ ]:
response, context = await api.global_search(
    config=graphrag_config,
    entities=entities_df,
    communities=communities_df,
    community_reports=community_reports_df,
    community_level=2,
    dynamic_community_selection=False,
    response_type="Multiple Paragraphs",
    query="",
)

In [11]:
print(response)

# Azure OpenAIで利用可能なモデル一覧と特徴

Azure OpenAIでは、最新かつ多様なAIモデルが利用可能です。これらのモデルは、テキスト生成、画像・音声処理、コード生成、マルチモーダル推論など、幅広い用途に対応しています。以下に、主要なモデル群とその特徴、利用可能なリージョン、アクセス要件などをまとめます。

---

## 1. 主なモデルファミリーと用途

### GPTシリーズ
- **GPT-3.5-Turbo, GPT-4, GPT-4 Turbo, GPT-4.1, GPT-4o, GPT-4o-mini, GPT-4.1-mini, GPT-35-Turbo, GPT-4-32K, GPT-40, GPT-40 Mini**
  - 高度な自然言語処理、長文対応、マルチモーダル（テキスト・画像）処理、リアルタイム音声処理など多様な機能を持つ [Data: Reports (49, 9, 0, 2, 79, 84, 27, 23, 61, +more)]。
  - GPT-4 TurboやGPT-4oは高速処理と大規模トークン制限に対応 [Data: Reports (43, 80, 79)]。
  - GPT-35-Turboはテキストベースのタスクに広く利用され、特定リージョンでファインチューニングも可能 [Data: Reports (46, 9)]。

### GPT-5ファミリー
- **GPT-5, GPT-5-CHAT, GPT-5-MINI, GPT-5-NANO, GPT-5-CODEX, GPT-5-PRO, GPT-5-5**
  - 会話、コード生成、軽量・高速処理、リアルタイム音声、構造化出力、外部ツール連携など多様な用途に最適化 [Data: Reports (4, 10, 54, 76, 31, 30)]。
  - GPT-5-MINIとGPT-5-NANOは登録不要で利用可能、GPT-5-CODEXやGPT-5-PROは登録が必要 [Data: Reports (76, 31, 30)]。
  - 最大400,000トークンのコンテキストウィンドウや128,000トークンの出力など、大規模な処理が可能 [Data: Reports (76, 31)]。

### GPT-5.1ファミリー
- **GPT

## DRIFT Search の仕組み

1. ユーザーのクエリが入力されると、まず **“コミュニティ要約 (Community Report)”** の集合の中から、クエリと意味的に最も関連性の高い上位 K 件を選び出し、**初期回答＋フォローアップ質問** を生成する。  
2. 生成されたフォローアップ質問に対しては、**Local Search の仕組み**を使って、より詳細な情報の探索と中間回答の収集を行う。  
3. フォローアップ→Local Search→さらに質問生成、というサイクルを、あらかじめ定められた条件 (繰り返し回数やスコア閾値) に基づいて繰り返すことで、情報の深掘りと精度向上を図る。  
4. 最終的に、Global 的視点（コミュニティ要約ベース）と Local 的視点（テキストチャンク＋エンティティ構造ベース）が統合された、**階層構造つきの質問／回答群** を生成する。  
5. このアプローチにより、広範囲な知識の「俯瞰 (global)」と、個別情報の「精査 (local)」を両立させ、**多様かつ文脈に即した回答** を得ることが可能になる。  

参考：https://microsoft.github.io/graphrag/query/drift_search/

In [19]:
response, context = await api.drift_search(
    config=graphrag_config,
    entities=entities_df,
    communities=communities_df,
    community_reports=community_reports_df,
    text_units=text_units_df,
    relationships=relationships_df,
    community_level=2,
    response_type="Multiple Paragraphs",
    query="Azure OpenAIで使えるモデルを教えて",
)

/home/taishi/work/176_graphrag_workshop/graphrag-workshop/.venv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
                                               

In [21]:
print(response)

# Azure OpenAIで利用可能な主なモデル一覧

Azure OpenAIでは、Microsoft Azureのクラウドインフラ上で多様なAIモデルが利用可能です。以下に、2025年時点で利用できる主なモデル群とその特徴をまとめます。

---

## 1. GPT-5シリーズ

- **gpt-5, gpt-5-mini, gpt-5-nano, gpt-5-chat, gpt-5-pro, gpt-5-codex**
  - 最新世代の大規模言語モデル。最大40万トークンのコンテキストウィンドウ、12.8万トークンの出力に対応。
  - gpt-5-proやgpt-5-codexは登録が必要ですが、gpt-5-mini、gpt-5-nano、gpt-5-chatは登録不要で利用可能。
  - チャット、推論、コード生成、画像処理、関数呼び出し、構造化出力など多機能。
  - gpt-5-chatは感情的知性やメンタルヘルス支援に特化したバージョンもあり [Data: Sources (3, 2, 1, 4)+more]。

## 2. GPT-4シリーズ

- **GPT-4.1, GPT-4.1-mini, GPT-4.1-nano, GPT-4 Turbo, GPT-4o, GPT-4o-mini**
  - テキストと画像の入力に対応したマルチモーダルモデル。
  - 最大128,000～1,047,576トークンのコンテキストウィンドウ、最大32,768トークンの出力。
  - GPT-4oは英語以外の言語や画像認識タスクで高い性能を発揮 [Data: Sources (7, 10, 14, 22, 24)+more]。

## 3. oシリーズ

- **o1, o1-mini, o1-preview, o3, o3-mini, o3-pro, o4-mini**
  - 高度な推論や問題解決能力を強化したモデル群。
  - テキスト・画像入力、構造化出力、関数・ツール呼び出し、ストリーミングなどに対応。
  - o1-miniはグローバル標準展開で全てのお客様が利用可能 [Data: Sources (15, 19, 20)+more]。

## 4. Codexシリーズ

- **codex-mini, gpt-5.1-codex, gpt